In [13]:
import torch
import os
import numpy as np
import util
import data_util
import matplotlib.pyplot as plt
from glob import glob
from scipy.io import loadmat
import cv2
from skimage.measure import compare_psnr

from deep_voxels import DeepVoxels
from projection import ProjectionHelper

In [14]:
device = torch.device('cuda')
model_path = os.path.join('/home/max/cache/logs/', 'model-epoch_36_iter_24000.pth')
data_path = os.path.join('/home/max/depth_infer_dataset/')

In [15]:
img_dim = 256
grid_dim = 32
proj_image_dims = [64, 64]

_, grid_barycenter, scale, near_plane, _ = \
    util.parse_intrinsics(os.path.join(data_path, 'intrinsics.txt'), trgt_sidelength=proj_image_dims[0])

if near_plane == 0.0:
    near_plane = np.sqrt(3)/2

In [16]:
lift_intrinsic = util.parse_intrinsics(os.path.join(data_path, 'intrinsics.txt'),
                                       trgt_sidelength=proj_image_dims[0])[0]
proj_intrinsic = lift_intrinsic

# Set up scale and world coordinates of voxel grid
voxel_size = (1. / grid_dim) * 1.1 * scale
grid_origin = torch.tensor(np.eye(4)).float().to(device).squeeze()
grid_origin[:3,3] = grid_barycenter

# Minimum and maximum depth used for rejecting voxels outside of the cmaera frustrum
depth_min = 0.
depth_max = grid_dim * voxel_size + near_plane
grid_dims = 3 * [grid_dim]

# Resolution of canonical viewing volume in the depth dimension, in number of voxels.
frustrum_depth = int(np.ceil(1.5 * grid_dim))

In [17]:
model = DeepVoxels(lifting_img_dims=proj_image_dims,
                   frustrum_img_dims=proj_image_dims,
                   grid_dims=grid_dims,
                   use_occlusion_net=True,
                   num_grid_feats=64,
                   nf0=64,
                   img_sidelength=img_dim)
model.to(device)


# Projection module
projection = ProjectionHelper(projection_intrinsic=proj_intrinsic,
                              lifting_intrinsic=lift_intrinsic,
                              depth_min=depth_min,
                              depth_max=depth_max,
                              projection_image_dims=proj_image_dims,
                              lifting_image_dims=proj_image_dims,
                              grid_dims=grid_dims,
                              voxel_size=voxel_size,
                              device=device,
                              frustrum_depth=frustrum_depth,
                              near_plane=near_plane)

OcclusionNet(
  (occlusion_prep): Sequential(
    (0): Conv3dSame(
      (net): Sequential(
        (0): ReplicationPad3d((1, 1, 1, 1, 1, 1))
        (1): Conv3d(65, 4, kernel_size=(3, 3, 3), stride=(1, 1, 1), bias=False)
      )
    )
    (1): BatchNorm3d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
  )
  (occlusion_net): Unet3d(
    (in_layer): Sequential(
      (0): Conv3dSame(
        (net): Sequential(
          (0): ReplicationPad3d((1, 1, 1, 1, 1, 1))
          (1): Conv3d(4, 4, kernel_size=(3, 3, 3), stride=(1, 1, 1), bias=False)
        )
      )
      (1): BatchNorm3d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2, inplace)
    )
    (unet_block): UnetSkipConnectionBlock3d(
      (model): Sequential(
        (0): DownBlock3D(
          (net): Sequential(
            (0): ReplicationPad3d((1, 1, 1, 1, 1, 1))
            (1): Conv3d(4, 8, kernel_size=(4, 4, 4), stride=(2, 2,


****************************************************************************************************
Lifting intrinsic is [[120.   0.  32.   0.]
 [  0. 120.  32.   0.]
 [  0.   0.   1.   0.]
 [  0.   0.   0.   1.]]
Projection intrinsic is [[120.   0.  32.   0.]
 [  0. 120.  32.   0.]
 [  0.   0.   1.   0.]
 [  0.   0.   0.   1.]]
Lifting image dims is  [64, 64]
Projection image dims is  [64, 64]
voxel size is 0.034375
****************************************************************************************************



In [18]:
util.custom_load(model, model_path)
model.eval()

DeepVoxels(
  (feature_extractor): Sequential(
    (0): DownsamplingNet(
      (downs): Sequential(
        (0): DownBlock(
          (net): Sequential(
            (0): ReflectionPad2d((1, 1, 1, 1))
            (1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
            (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (3): LeakyReLU(negative_slope=0.2, inplace)
            (4): ReflectionPad2d((1, 1, 1, 1))
            (5): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), bias=False)
            (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (7): LeakyReLU(negative_slope=0.2, inplace)
          )
        )
        (1): DownBlock(
          (net): Sequential(
            (0): ReflectionPad2d((1, 1, 1, 1))
            (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
            (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [19]:
rgb_list = sorted(glob(os.path.join(data_path,'rgb','*.png')))
pose_list = sorted(glob(os.path.join(data_path,'pose','*.mat')))
depth_list = sorted(glob(os.path.join(data_path,'depth','*.png')))

In [20]:
psnr = list()
for i in range(len(rgb_list)//2):
    img1 = data_util.load_img(rgb_list[2*i], square_crop=True, downsampling_order=1, target_size=[img_dim, img_dim])
    img1 = img1[:, :, :3].astype(np.float32) / 255. - 0.5
    img1 = img1.transpose(2,0,1)
    img1 = torch.from_numpy(img1)
    img2 = data_util.load_img(rgb_list[2*i+1], square_crop=True, downsampling_order=1, target_size=[img_dim, img_dim])
    img2 = img2[:, :, :3].astype(np.float32) / 255. - 0.5
    img2 = img2.transpose(2,0,1)
    img2 = torch.from_numpy(img2)

    pose1 = loadmat(pose_list[2*i])['extrinsic']
    pose1 = np.linalg.inv(pose1).astype(np.float32)
    pose1 = torch.from_numpy(pose1)
    pose2 = loadmat(pose_list[2*i+1])['extrinsic']
    pose2 = np.linalg.inv(pose2).astype(np.float32)
    pose2 = torch.from_numpy(pose2)

    num_pretest = 1
    with torch.no_grad():
        model.representation = torch.zeros((1, model.n_grid_feats, model.grid_dims[0], model.grid_dims[1], model.grid_dims[2])).cuda()
        for num in range(num_pretest):
            backproj_mapping = list()
            inpt_rgbs = list()
            backproj_mapping.append(projection.comp_lifting_idcs(camera_to_world=pose1.squeeze().to(device),grid2world=grid_origin))
            backproj_mapping.append(projection.comp_lifting_idcs(camera_to_world=pose2.squeeze().to(device),grid2world=grid_origin))

            inpt_rgbs.append(img1.unsqueeze(0).to(device)) 
            inpt_rgbs.append(img2.unsqueeze(0).to(device)) 

            lift_volume_idcs, lift_img_coords = list(zip(*backproj_mapping))

            model(inpt_rgbs, 'pretest', None, None, lift_volume_idcs, lift_img_coords, None)

        trgt_pose = pose1.squeeze().to(device)

        # compute projection mapping
        proj_mapping = projection.compute_proj_idcs(trgt_pose.squeeze(), grid_origin)

        proj_ind_3d, proj_ind_2d = proj_mapping

        # Run through model
        _, depth_maps = model(None, 'test', [proj_ind_3d], [proj_ind_2d], None, None, None)

        depth_img = depth_maps[0].squeeze(0).cpu().detach().numpy()
        depth_img = depth_img.transpose(1, 2, 0)
        depth_img = (depth_img - np.amin(depth_img)) / (np.amax(depth_img) - np.amin(depth_img))
        depth_img = depth_img.squeeze()
        
        true_depth = -cv2.imread(depth_list[2*i], cv2.IMREAD_UNCHANGED)
        true_depth = cv2.resize(true_depth, tuple(proj_image_dims), interpolation=cv2.INTER_AREA)
        true_depth = (true_depth - np.amin(true_depth)) / (np.amax(true_depth) - np.amin(true_depth))
        true_depth = true_depth.astype(np.float32)
        
        psnr.append(compare_psnr(true_depth, depth_img))
        
        if i%10 == 0:
            print(i)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290


In [ ]:
plt.imshow(depth_img.squeeze().astype(np.uint16), cmap='Greys')

In [21]:
max(psnr)

13.590636132291165

In [22]:
min(psnr)

3.2449079046405602

In [23]:
np.save('/home/max/saved_data/binocular_psnr', np.array(psnr))

In [26]:
np.load('/home/max/saved_data/binocular_psnr.npy').max()

13.590636132291165